In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Reasoning Engine in Vertex AI
This lab simplified the original colab [intro_reasoning_engine](https://colab.sandbox.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/reasoning-engine/intro_reasoning_engine.ipynb) |Author(s) | [Kristopher Overholt](https://github.com/koverholt) |


### Reasoning Engine in Vertex AI

[Reasoning Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/overview) (LangChain on Vertex AI) is a managed service that helps you to build and deploy an agent reasoning framework. It gives you the flexibility to choose how much reasoning you want to delegate to the LLM and how much you want to handle with customized code. You can define Python functions that get used as tools via [Gemini Function Calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling). Reasoning Engine integrates closely with the Python SDK for the Gemini model in Vertex AI, and it can manage prompts, agents, and examples in a modular way. Reasoning Engine is compatible with LangChain, LlamaIndex, or other Python frameworks.


### Install Vertex AI SDK for Python

In [3]:
!pip install --upgrade --user --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

In [ ]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Initial set up

In [ ]:
# @title Define constants
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash-002" # @param {type:"string"}

In [ ]:
# @title Create a bucket.
BUCKET_URI = f"gs://mlops-{PROJECT_ID}-1209"
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://mlops-ai-hangsik-1209/...
ServiceException: 409 A Cloud Storage bucket named 'mlops-ai-hangsik-1209' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [ ]:
# @title Service account
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")

SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

SERVICE_ACCOUNT: 721521243942-compute@developer.gserviceaccount.com


In [ ]:
# @title Set access to the bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://mlops-ai-hangsik-1209/
No changes made to gs://mlops-ai-hangsik-1209/


In [ ]:
# @title Initialize Vertex AI with Staging Bucket.

import vertexai
from vertexai.preview import reasoning_engines

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

In [ ]:
# @title Tool define
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [ ]:
# Tool test
get_exchange_rate(currency_from="USD", currency_to="KRW")

{'amount': 1.0, 'base': 'USD', 'date': '2024-12-27', 'rates': {'KRW': 1473.55}}

In [ ]:
# @title Agent Define - Local unit test
agent = reasoning_engines.LangchainAgent(
    model=MODEL_NAME,
    tools=[get_exchange_rate],
    agent_executor_kwargs={"return_intermediate_steps": False},
)

In [ ]:
# @title Test your agent locally

# Agent local unit test before deploying.
agent.query(input="What's the exchange rate from US dollars to Korean currency today?")

{'input': "What's the exchange rate from US dollars to Korean currency today?",
 'output': "Today's exchange rate is 1 USD to 1473.55 KRW.\n"}

In [ ]:
# @title Deploy your agent on Vertex AI

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.ReasoningEngine#vertexai_preview_reasoning_engines_ReasoningEngine_create

remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    display_name="simple agent",
    reasoning_engine_name="ai-agent", # does not support user-defined resource IDs at the moment
    gcs_dir_name = "ai-agent-1",
    description="This is a simple reasoning engine.",
    requirements=[
        "google-cloud-aiplatform[langchain,reasoningengine]",
        "cloudpickle==3.0.0",
        "pydantic==2.7.4",
        "requests",
    ],
    extra_packages = []
)

INFO:vertexai.reasoning_engines._reasoning_engines:Using bucket mlops-ai-hangsik-1209
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://mlops-ai-hangsik-1209/ai-agent-1/reasoning_engine.pkl
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://mlops-ai-hangsik-1209/ai-agent-1/requirements.txt
INFO:vertexai.reasoning_engines._reasoning_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.reasoning_engines._reasoning_engines:Writing to gs://mlops-ai-hangsik-1209/ai-agent-1/dependencies.tar.gz
INFO:vertexai.reasoning_engines._reasoning_engines:Creating ReasoningEngine
INFO:vertexai.reasoning_engines._reasoning_engines:Create ReasoningEngine backing LRO: projects/721521243942/locations/us-central1/reasoningEngines/7298417447688732672/operations/1428695960204083200
INFO:vertexai.reasoning_engines._reasoning_engines:ReasoningEngine created. Resource name: projects/721521243942/locations/us-central1/reasoningEngines/7298417447688732672
INFO:vertexai

In [ ]:
# @title Query from remote engine.
remote_agent.query(
    input="What's the exchange rate from US dollars to Korea won  currency today?"
)

{'input': "What's the exchange rate from US dollars to Korea won  currency today?",
 'output': "Today's exchange rate is 1 USD to 1473.55 KRW."}

In [ ]:
# @title Helper function to manage reasoning engine.

# Properties of ReasoningEngine class.
# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.ReasoningEngine

#----------------------------------------
def get_reasoning_engine(display_name:str):
  try:
    for re in reasoning_engines.ReasoningEngine.list():
      if re.display_name == display_name:
        return reasoning_engines.ReasoningEngine(re.name)
      else:
        print("No such reasoning engine or Invalid display name.")

  except Exception as e:
    print(e)

#----------------------------------------
def get_re_list():
  """
  List reasoning engines.
  """

  try:
    if not reasoning_engines.ReasoningEngine.list():
      print("No reasoning engines")

    for idx, re in enumerate(reasoning_engines.ReasoningEngine.list()):
        print(f"Agent {idx}: \n\tDisplay Name [{re.display_name}] \n\tName [{re.name}] \n\tCreation Time [{re.create_time}] \n\tResource Name [{re.resource_name}]\n")

  except Exception as e:
    print(e)

#----------------------------------------

def del_re(name):
  """
  Delete a reasoning engines.
  @param name: The name of the reasoning engine.
  @type name: str
  """

  try:
    re = reasoning_engines.ReasoningEngine(name)
    re.delete()
    print(f"Deleted {name}")
  except Exception as e:
    print(e)


In [ ]:
get_re_list()

Agent 0: 
	Display Name [simple agent] 
	Name [7298417447688732672] 
	Creation Time [2024-12-27 18:12:51.870042+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/7298417447688732672]

Agent 1: 
	Display Name [simple agent] 
	Name [634497374063951872] 
	Creation Time [2024-12-17 22:12:18.771061+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/634497374063951872]

Agent 2: 
	Display Name [multi tool agent] 
	Name [1508653098610982912] 
	Creation Time [2024-12-08 14:47:51.100096+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/1508653098610982912]

Agent 3: 
	Display Name [multi tool agent] 
	Name [5120821474738831360] 
	Creation Time [2024-11-19 01:33:51.712140+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/5120821474738831360]

Agent 4: 
	Display Name [simple agent] 
	Name [6325534375060439040] 
	Creation Time [2024-11-18 13:34:12.086807+00:00]

In [ ]:
# @title Get instances with display name.
re = get_reasoning_engine('simple agent')
print(re)
re.query(
    input="What's the exchange rate from US dollars to Korea won currency today?"
)

resource name: projects/ai-hangsik/locations/us-central1/reasoningEngines/7298417447688732672


{'input': "What's the exchange rate from US dollars to Korea won currency today?",
 'output': 'The exchange rate from US dollars (USD) to Korea won (KRW) today, December 27th 2024, is 1473.55 KRW per 1 USD.\n'}